# Enunciado

1. Ainda com relação à lista 08, faça um grid search sobre o conjunto de dados escalonado com o StandardScaler para comparar o RMSE das seguintes implementações do Scikit Learn:
  * LinearRegression
  * SGDRegressor (teste alguns valores dos seguintes hiper-parâmetros: penalty, alpha, max_iter, early_stopping)
  * Ridge (teste alguns valores dos seguintes hiper-parâmetros: alpha, max_iter)
  * Lasso (teste alguns valores dos seguintes hiper-parâmetros: alpha, max_iter)
  * ElasticNet (teste alguns valores dos seguintes hiper-parâmetros: alpha, l1_ratio, max_iter)
2. Implemente o algoritmo Regressão Logística usando uma interface semelhante ao Scikit-Learn. O algoritmo deve ser uma classe Python em uma biblioteca externa.
3. Treine e avalie (de acordo com a métrica F1-Score), usando a Regressão Logística implementada por você, o problema de classificação binária a seguir: Heart Disease Dataset
4. Compare o resultado de sua implementação com a implementação LogisticRegression do scikit learn em um grid search, que varia para a implementação do scikit learn, os seguintes hiper-parâmetros: penalty, C, solver, max_iter.

# Importações

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing

#auxiliares
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

#metricas
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score

#implementações para comparar com minhas libs
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [2]:
#minhas libs
from libs import metrics
from libs import kneighbors_regression
from libs import simple_linear_regression

In [3]:
housing = fetch_california_housing()

In [4]:
y = housing.target[0:2000]
x = housing.data[0:2000, 2].reshape(-1, 1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=.8, random_state=42)

In [6]:
clf1 = simple_linear_regression.SimpleLinearRegression()
clf2 = kneighbors_regression.KNeighborsRegression()
clf3 = LinearRegression()
clf4 = SGDRegressor(random_state=42)
clf5 = Ridge(random_state=42)
clf6 = Lasso(random_state=42)
clf7 = ElasticNet(random_state=42)

In [7]:
classifiers = [clf1, clf2, clf3, clf4, clf5, clf6, clf7]

In [8]:
clf4.get_params()

{'alpha': 0.0001,
 'average': False,
 'early_stopping': False,
 'epsilon': 0.1,
 'eta0': 0.01,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'invscaling',
 'loss': 'squared_error',
 'max_iter': 1000,
 'n_iter_no_change': 5,
 'penalty': 'l2',
 'power_t': 0.25,
 'random_state': 42,
 'shuffle': True,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [9]:
param1 = {}
param1['classifier'] = [clf1]

param2 = {}
param2['classifier'] = [clf2]

param3 = {}
param3['classifier'] = [clf3]

param4 = {}
param4['classifier__penalty'] = ['l2', 'l1', 'elasticnet']
param4['classifier__alpha'] = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
param4['classifier__max_iter'] = [500, 1000, 2000]
param4['classifier'] = [clf4]

param5 = {}
param5['classifier__alpha'] = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
param5['classifier__max_iter'] = [500, 1000, 2000]
param5['classifier'] = [clf5]

param6 = {}
param6['classifier__alpha'] = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
param6['classifier__max_iter'] = [500, 1000, 2000]
param6['classifier'] = [clf6]

param7 = {}
param7['classifier__alpha'] = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
param7['classifier__l1_ratio'] = [0.5, 0.4, 0.8]
param7['classifier__max_iter'] = [500, 1000, 2000]
param7['classifier'] = [clf7]

In [10]:
params = [param1, param2, param3, param4, param5, param6, param7]

In [11]:
pipeline = Pipeline([('standard', StandardScaler()), ('classifier', clf1)])

In [12]:
gs1 = GridSearchCV(pipeline, params, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', return_train_score=True).fit(X_train, y_train)

In [13]:
gs1.best_estimator_

Pipeline(steps=[('standard', StandardScaler()),
                ('classifier',
                 Ridge(alpha=100.0, max_iter=500, random_state=42))])

In [14]:
gs1.best_score_

-0.9212413475574831

In [15]:
y_pred_gs1 = gs1.predict(X_test)

In [16]:
mean_squared_error(y_test, y_pred_gs1)

0.890751135028763

In [17]:
df_results_gs1 = pd.DataFrame(gs1.cv_results_)
df_results_gs1.shape

(195, 21)

In [18]:
-0.92 < -0.93

False

In [19]:
df_results_gs1 = df_results_gs1.sort_values(by='rank_test_score')
df_results_gs1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__alpha,param_classifier__max_iter,param_classifier__penalty,param_classifier__l1_ratio,params,...,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
95,0.001741,0.000065,0.000648,0.000019,"Ridge(alpha=100.0, max_iter=500, random_state=42)",100.0,2000,NaN,NaN,"{'classifier': Ridge(alpha=100.0, max_iter=500...",...,-0.857557,-0.930896,-0.921241,0.048539,1,-0.884789,-0.943054,-0.906699,-0.911514,0.024029
94,0.001662,0.000073,0.000629,0.000050,"Ridge(alpha=100.0, max_iter=500, random_state=42)",100.0,1000,NaN,NaN,"{'classifier': Ridge(alpha=100.0, max_iter=500...",...,-0.857557,-0.930896,-0.921241,0.048539,1,-0.884789,-0.943054,-0.906699,-0.911514,0.024029
93,0.002101,0.000526,0.000665,0.000029,"Ridge(alpha=100.0, max_iter=500, random_state=42)",100.0,500,NaN,NaN,"{'classifier': Ridge(alpha=100.0, max_iter=500...",...,-0.857557,-0.930896,-0.921241,0.048539,1,-0.884789,-0.943054,-0.906699,-0.911514,0.024029
154,0.002142,0.000046,0.000684,0.000015,ElasticNet(random_state=42),0.1,1000,NaN,0.4,"{'classifier': ElasticNet(random_state=42), 'c...",...,-0.862664,-0.935196,-0.921308,0.043340,4,-0.886821,-0.944923,-0.908632,-0.913459,0.023964
155,0.002206,0.000126,0.000675,0.000015,ElasticNet(random_state=42),0.1,2000,NaN,0.4,"{'classifier': ElasticNet(random_state=42), 'c...",...,-0.862664,-0.935196,-0.921308,0.043340,4,-0.886821,-0.944923,-0.908632,-0.913459,0.023964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.003550,0.000107,0.000684,0.000004,SGDRegressor(random_state=42),1000.0,500,l1,NaN,"{'classifier': SGDRegressor(random_state=42), ...",...,-0.887859,-0.947421,-0.935626,0.035189,171,-0.913752,-0.956773,-0.925417,-0.931981,0.018166
66,0.003612,0.000111,0.002167,0.002051,SGDRegressor(random_state=42),1000.0,500,l2,NaN,"{'classifier': SGDRegressor(random_state=42), ...",...,-0.887862,-0.947416,-0.935626,0.035188,192,-0.913754,-0.956773,-0.925409,-0.931979,0.018167
69,0.003621,0.000123,0.000657,0.000039,SGDRegressor(random_state=42),1000.0,1000,l2,NaN,"{'classifier': SGDRegressor(random_state=42), ...",...,-0.887862,-0.947416,-0.935626,0.035188,192,-0.913754,-0.956773,-0.925409,-0.931979,0.018167
72,0.008730,0.003958,0.000763,0.000156,SGDRegressor(random_state=42),1000.0,2000,l2,NaN,"{'classifier': SGDRegressor(random_state=42), ...",...,-0.887862,-0.947416,-0.935626,0.035188,192,-0.913754,-0.956773,-0.925409,-0.931979,0.018167


In [20]:
df_results_gs1.reset_index(drop=True, inplace=True)
df_results_gs1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__alpha,param_classifier__max_iter,param_classifier__penalty,param_classifier__l1_ratio,params,...,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.001741,0.000065,0.000648,0.000019,"Ridge(alpha=100.0, max_iter=500, random_state=42)",100.0,2000,NaN,NaN,"{'classifier': Ridge(alpha=100.0, max_iter=500...",...,-0.857557,-0.930896,-0.921241,0.048539,1,-0.884789,-0.943054,-0.906699,-0.911514,0.024029
1,0.001662,0.000073,0.000629,0.000050,"Ridge(alpha=100.0, max_iter=500, random_state=42)",100.0,1000,NaN,NaN,"{'classifier': Ridge(alpha=100.0, max_iter=500...",...,-0.857557,-0.930896,-0.921241,0.048539,1,-0.884789,-0.943054,-0.906699,-0.911514,0.024029
2,0.002101,0.000526,0.000665,0.000029,"Ridge(alpha=100.0, max_iter=500, random_state=42)",100.0,500,NaN,NaN,"{'classifier': Ridge(alpha=100.0, max_iter=500...",...,-0.857557,-0.930896,-0.921241,0.048539,1,-0.884789,-0.943054,-0.906699,-0.911514,0.024029
3,0.002142,0.000046,0.000684,0.000015,ElasticNet(random_state=42),0.1,1000,NaN,0.4,"{'classifier': ElasticNet(random_state=42), 'c...",...,-0.862664,-0.935196,-0.921308,0.043340,4,-0.886821,-0.944923,-0.908632,-0.913459,0.023964
4,0.002206,0.000126,0.000675,0.000015,ElasticNet(random_state=42),0.1,2000,NaN,0.4,"{'classifier': ElasticNet(random_state=42), 'c...",...,-0.862664,-0.935196,-0.921308,0.043340,4,-0.886821,-0.944923,-0.908632,-0.913459,0.023964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.003550,0.000107,0.000684,0.000004,SGDRegressor(random_state=42),1000.0,500,l1,NaN,"{'classifier': SGDRegressor(random_state=42), ...",...,-0.887859,-0.947421,-0.935626,0.035189,171,-0.913752,-0.956773,-0.925417,-0.931981,0.018166
191,0.003612,0.000111,0.002167,0.002051,SGDRegressor(random_state=42),1000.0,500,l2,NaN,"{'classifier': SGDRegressor(random_state=42), ...",...,-0.887862,-0.947416,-0.935626,0.035188,192,-0.913754,-0.956773,-0.925409,-0.931979,0.018167
192,0.003621,0.000123,0.000657,0.000039,SGDRegressor(random_state=42),1000.0,1000,l2,NaN,"{'classifier': SGDRegressor(random_state=42), ...",...,-0.887862,-0.947416,-0.935626,0.035188,192,-0.913754,-0.956773,-0.925409,-0.931979,0.018167
193,0.008730,0.003958,0.000763,0.000156,SGDRegressor(random_state=42),1000.0,2000,l2,NaN,"{'classifier': SGDRegressor(random_state=42), ...",...,-0.887862,-0.947416,-0.935626,0.035188,192,-0.913754,-0.956773,-0.925409,-0.931979,0.018167


# Comparação - Quest 1

In [21]:
scores = list()
for classifier in classifiers:
  print(classifier)
  query = df_results_gs1[df_results_gs1.param_classifier.eq(classifier)].iloc[0,:]
  res = query[['mean_train_score', 'mean_test_score']].values * (-1)
  print('train:', res[0], 'test:', res[1])
  scores.append(res)

train: 0.9319549146198193 test: 0.9352111642941887
train: 5.343220152993126 test: 5.533176566799983
LinearRegression()
train: 0.9113627923607445 test: 0.9223569396667157
SGDRegressor(random_state=42)
train: 0.9162360715841698 test: 0.921927531159242
Ridge(alpha=100.0, max_iter=500, random_state=42)
train: 0.9115141172891735 test: 0.9212413475574831
Lasso(random_state=42)
train: 0.9114627923607445 test: 0.9217179911775778
ElasticNet(random_state=42)
train: 0.9134585989969987 test: 0.9213078206725487


In [22]:
scores

[array([0.9319549146198193, 0.9352111642941887], dtype=object),
 array([5.343220152993126, 5.533176566799983], dtype=object),
 array([0.9113627923607445, 0.9223569396667157], dtype=object),
 array([0.9162360715841698, 0.921927531159242], dtype=object),
 array([0.9115141172891735, 0.9212413475574831], dtype=object),
 array([0.9114627923607445, 0.9217179911775778], dtype=object),
 array([0.9134585989969987, 0.9213078206725487], dtype=object)]

In [23]:
classifiers

 LinearRegression(),
 SGDRegressor(random_state=42),
 Ridge(alpha=100.0, max_iter=500, random_state=42),
 Lasso(random_state=42),
 ElasticNet(random_state=42)]

In [24]:
ranking = pd.DataFrame(columns=['model', 'mean_train_score', 'mean_test_score'])
for i in range(len(classifiers)):
  ranking.loc[i,:] = [classifiers[i], scores[i][0], scores[i][1]]

In [25]:
ranking.sort_values(by='mean_train_score')

,model,mean_train_score,mean_test_score
2,LinearRegression(),0.911363,0.922357
5,Lasso(random_state=42),0.911463,0.921718
4,"Ridge(alpha=100.0, max_iter=500, random_state=42)",0.911514,0.921241
6,ElasticNet(random_state=42),0.913459,0.921308
3,SGDRegressor(random_state=42),0.916236,0.921928
0,<libs.simple_linear_regression.SimpleLinearReg...,0.931955,0.935211
1,<libs.kneighbors_regression.KNeighborsRegressi...,5.34322,5.533177


In [26]:
ranking.sort_values(by='mean_test_score')

,model,mean_train_score,mean_test_score
4,"Ridge(alpha=100.0, max_iter=500, random_state=42)",0.911514,0.921241
6,ElasticNet(random_state=42),0.913459,0.921308
5,Lasso(random_state=42),0.911463,0.921718
3,SGDRegressor(random_state=42),0.916236,0.921928
2,LinearRegression(),0.911363,0.922357
0,<libs.simple_linear_regression.SimpleLinearReg...,0.931955,0.935211
1,<libs.kneighbors_regression.KNeighborsRegressi...,5.34322,5.533177


# Quest 2
Implemente o algoritmo Regressão Logística usando uma interface semelhante ao Scikit-Learn. O algoritmo deve ser uma classe Python em uma biblioteca externa.

In [27]:
from libs import logit_regression

# Quest 3
Treine e avalie (de acordo com a métrica F1-Score), usando a Regressão Logística implementada por você, o problema de classificação binária a seguir: Heart Disease Dataset

In [28]:
df = pd.read_csv('/content/drive/MyDrive/machine-learning/heart.csv')

In [29]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [30]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [31]:
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=.8, random_state=42)

In [32]:
s = StandardScaler()
X_train = s.fit_transform(X_train)

In [33]:
model = logit_regression.BinayLogitRegression(learning_rate = 0.01, iterations = 1000)

In [34]:
model.fit(X_train, y_train)

In [35]:
#(None, 'micro', 'macro', 'weighted', 'samples')
pred = model.predict(X_test)
print("Test F1-Score:", f1_score(y_test, pred, average=None))
print("Test F1-Score:", f1_score(y_test, pred, average='micro'))
print("Test F1-Score:", f1_score(y_test, pred, average='macro'))
print("Test F1-Score:", f1_score(y_test, pred, average='weighted'))

Test F1-Score: [0.68592058 0.34586466]
Test F1-Score: 0.5756097560975609
Test F1-Score: 0.515892619635732
Test F1-Score: 0.5150632149626511


# Quest 4
Compare o resultado de sua implementação com a implementação LogisticRegression do scikit learn em um grid search, que varia para a implementação do scikit learn, os seguintes hiper-parâmetros: penalty, C, solver, max_iter.

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
clf1 = logit_regression.BinayLogitRegression(learning_rate = 0.01, iterations = 1000)
clf2 = LogisticRegression()
classifiers = [clf1, clf2]

In [38]:
param1 = {}
param1['classifier'] = [clf1]

param2 = {}
param2['classifier__C'] = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
param2['classifier__max_iter'] = [500, 1000, 2000]
param2['classifier'] = [clf2]

In [39]:
params = [param1, param2]

In [40]:
pipeline = Pipeline([('standard', StandardScaler()), ('classifier', clf1)])

In [41]:
gs2 = GridSearchCV(pipeline, params, cv=3, n_jobs=-1, scoring='f1_micro', return_train_score=True).fit(X_train, y_train)

In [42]:
df_results_gs2 = pd.DataFrame(gs2.cv_results_)
df_results_gs2.shape

(25, 19)

In [43]:
df_results_gs2 = df_results_gs2.sort_values(by='rank_test_score')
df_results_gs2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__C,param_classifier__max_iter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
12,0.011515,0.005834,0.001914,0.000052,"LogisticRegression(C=0.1, max_iter=500)",0.1,2000,"{'classifier': LogisticRegression(C=0.1, max_i...",0.828467,0.890110,0.824176,0.847584,0.030121,1,0.866300,0.851920,0.890311,0.869510,0.015837
11,0.007427,0.000163,0.001837,0.000024,"LogisticRegression(C=0.1, max_iter=500)",0.1,1000,"{'classifier': LogisticRegression(C=0.1, max_i...",0.828467,0.890110,0.824176,0.847584,0.030121,1,0.866300,0.851920,0.890311,0.869510,0.015837
10,0.007316,0.000096,0.001798,0.000040,"LogisticRegression(C=0.1, max_iter=500)",0.1,500,"{'classifier': LogisticRegression(C=0.1, max_i...",0.828467,0.890110,0.824176,0.847584,0.030121,1,0.866300,0.851920,0.890311,0.869510,0.015837
0,0.098362,0.005065,0.002831,0.000461,<libs.logit_regression.BinayLogitRegression ob...,NaN,NaN,{'classifier': <libs.logit_regression.BinayLog...,0.843066,0.875458,0.816850,0.845124,0.023971,4,0.860806,0.855576,0.882998,0.866460,0.011888
14,0.008546,0.000944,0.003545,0.002438,"LogisticRegression(C=0.1, max_iter=500)",1.0,1000,"{'classifier': LogisticRegression(C=0.1, max_i...",0.824818,0.882784,0.824176,0.843926,0.027478,5,0.860806,0.846435,0.886654,0.864632,0.016641
15,0.007396,0.000152,0.001780,0.000034,"LogisticRegression(C=0.1, max_iter=500)",1.0,2000,"{'classifier': LogisticRegression(C=0.1, max_i...",0.824818,0.882784,0.824176,0.843926,0.027478,5,0.860806,0.846435,0.886654,0.864632,0.016641
13,0.007638,0.000337,0.001887,0.000013,"LogisticRegression(C=0.1, max_iter=500)",1.0,500,"{'classifier': LogisticRegression(C=0.1, max_i...",0.824818,0.882784,0.824176,0.843926,0.027478,5,0.860806,0.846435,0.886654,0.864632,0.016641
22,0.008667,0.000865,0.002614,0.000834,"LogisticRegression(C=0.1, max_iter=500)",1000.0,500,"{'classifier': LogisticRegression(C=0.1, max_i...",0.824818,0.879121,0.824176,0.842705,0.025751,8,0.860806,0.842779,0.886654,0.863413,0.018007
21,0.008020,0.000325,0.001977,0.000130,"LogisticRegression(C=0.1, max_iter=500)",100.0,2000,"{'classifier': LogisticRegression(C=0.1, max_i...",0.824818,0.879121,0.824176,0.842705,0.025751,8,0.860806,0.842779,0.886654,0.863413,0.018007
20,0.009155,0.001510,0.001868,0.000090,"LogisticRegression(C=0.1, max_iter=500)",100.0,1000,"{'classifier': LogisticRegression(C=0.1, max_i...",0.824818,0.879121,0.824176,0.842705,0.025751,8,0.860806,0.842779,0.886654,0.863413,0.018007


In [44]:
df_results_gs2.reset_index(drop=True, inplace=True)

In [45]:
scores = list()
for classifier in classifiers:
  print(classifier)
  query = df_results_gs2[df_results_gs2.param_classifier.eq(classifier)].iloc[0,:]
  res = query[['mean_train_score', 'mean_test_score']].values
  print('train:', res[0], 'test:', res[1])
  scores.append(res)

train: 0.8664599670084131 test: 0.8451244619127832
LogisticRegression(C=0.1, max_iter=500)
train: 0.8695102378831812 test: 0.8475842891901285


In [46]:
scores

[array([0.8664599670084131, 0.8451244619127832], dtype=object),
 array([0.8695102378831812, 0.8475842891901285], dtype=object)]

In [47]:
classifiers

 LogisticRegression(C=0.1, max_iter=500)]

In [48]:
ranking2 = pd.DataFrame(columns=['model', 'mean_train_score', 'mean_test_score'])
for i in range(len(classifiers)):
  ranking2.loc[i,:] = [classifiers[i], scores[i][0], scores[i][1]]

In [49]:
ranking2.sort_values(by='mean_train_score')

,model,mean_train_score,mean_test_score
0,<libs.logit_regression.BinayLogitRegression ob...,0.86646,0.845124
1,"LogisticRegression(C=0.1, max_iter=500)",0.86951,0.847584


In [50]:
ranking2.sort_values(by='mean_test_score')

,model,mean_train_score,mean_test_score
0,<libs.logit_regression.BinayLogitRegression ob...,0.86646,0.845124
1,"LogisticRegression(C=0.1, max_iter=500)",0.86951,0.847584


# Fim